In [1]:
import os
import torch
from datasets import load_dataset, load_metric
from transformers import (
    PreTrainedTokenizerFast,
    Trainer, 
    TrainingArguments, 
    AutoModelForSequenceClassification,
    AutoTokenizer,
)
import sentencepiece as spm

In [2]:
translation_dataset = load_dataset("wmt14", "de-en", split="train[:1%]")
sentiment_dataset = load_dataset("amazon_polarity", split="train[:1%]") 

In [3]:
#Phase 2: Experiments (Fine-tuning Models)

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM  # For translation tasks, e.g., MarianMT

def load_custom_tokenizer(tokenizer_type="bpe"):
    if tokenizer_type == "bpe":
        # Load a fast tokenizer from files
        # This is a simplified example, in practice you'd wrap it with PreTrainedTokenizerFast
        return AutoTokenizer.from_pretrained("tokenizers/bpe", use_fast=True)
    elif tokenizer_type == "sp":
        return AutoTokenizer.from_pretrained("tokenizers/sp_unigram_hf", use_fast=True)
    elif tokenizer_type == "wp":
        return AutoTokenizer.from_pretrained("tokenizers/wp", use_fast=True)
    else:
        raise ValueError("Unsupported tokenizer type")

baseline_tokenizer = load_custom_tokenizer("wp")

In [4]:
split_dataset = sentiment_dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

def preprocess_function_sentiment(examples):
    texts = examples["content"]
    labels = examples["label"]
    tokenized_inputs = baseline_tokenizer(
        texts,
        truncation=True,
        padding="max_length",
        max_length=128
    )
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_train = train_dataset.map(preprocess_function_sentiment, batched=True)
tokenized_eval = eval_dataset.map(preprocess_function_sentiment, batched=True)

tokenized_train = tokenized_train.remove_columns(["content", "label"])
tokenized_eval = tokenized_eval.remove_columns(["content", "label"])

tokenized_train.set_format("torch")
tokenized_eval.set_format("torch")


Map:   0%|          | 0/28800 [00:00<?, ? examples/s]

Map:   0%|          | 0/7200 [00:00<?, ? examples/s]

In [5]:
# For translation tasks, you might use a MarianMT model or mBART, for sentiment XLM-R or mBERT.
device = torch.device("mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")
model_name = "bert-base-uncased" 
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2).to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
train_args = TrainingArguments(
    output_dir="checkpoints/sentiment_wp",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="logs",
    num_train_epochs=10,          
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    save_total_limit=2,
    load_best_model_at_end=True,
    fp16=(device.type == "cuda"),
    push_to_hub=False,
)

metric_accuracy = load_metric("accuracy")

/Users/sg/anaconda3/lib/python3.11/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/var/folders/x8/bnjqxpd164n045xxqsqmvx9w0000gn/T/ipykernel_74395/1175339046.py:15: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric_accuracy = load_metric("accuracy")
/Users/sg/anaconda3/lib/python3.11/site-packages/datasets/load.py:759: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=Tru

In [7]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    return metric_accuracy.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model=model,
    args=train_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    tokenizer=baseline_tokenizer,
    compute_metrics=compute_metrics
)

/var/folders/x8/bnjqxpd164n045xxqsqmvx9w0000gn/T/ipykernel_74395/1230184579.py:6: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [8]:
trainer.train()

  0%|          | 0/18000 [00:00<?, ?it/s]

{'loss': 0.7067, 'grad_norm': 1.5463447570800781, 'learning_rate': 4.8611111111111115e-05, 'epoch': 0.28}
{'loss': 0.7037, 'grad_norm': 2.660998821258545, 'learning_rate': 4.722222222222222e-05, 'epoch': 0.56}
{'loss': 0.7012, 'grad_norm': 11.847258567810059, 'learning_rate': 4.5833333333333334e-05, 'epoch': 0.83}


  0%|          | 0/450 [00:00<?, ?it/s]

{'eval_loss': 0.6944168210029602, 'eval_accuracy': 0.4915277777777778, 'eval_runtime': 242.2197, 'eval_samples_per_second': 29.725, 'eval_steps_per_second': 1.858, 'epoch': 1.0}
{'loss': 0.7003, 'grad_norm': 3.52889347076416, 'learning_rate': 4.4444444444444447e-05, 'epoch': 1.11}
{'loss': 0.6974, 'grad_norm': 1.674487829208374, 'learning_rate': 4.305555555555556e-05, 'epoch': 1.39}
{'loss': 0.6977, 'grad_norm': 2.19793701171875, 'learning_rate': 4.166666666666667e-05, 'epoch': 1.67}
{'loss': 0.6961, 'grad_norm': 1.1906877756118774, 'learning_rate': 4.027777777777778e-05, 'epoch': 1.94}


  0%|          | 0/450 [00:00<?, ?it/s]

...

  0%|          | 0/450 [00:00<?, ?it/s]

{'eval_loss': 0.6939027309417725, 'eval_accuracy': 0.5084722222222222, 'eval_runtime': 222.1538, 'eval_samples_per_second': 32.41, 'eval_steps_per_second': 2.026, 'epoch': 9.0}
{'loss': 0.6965, 'grad_norm': 3.3407113552093506, 'learning_rate': 4.166666666666667e-06, 'epoch': 9.17}
{'loss': 0.6969, 'grad_norm': 4.7902960777282715, 'learning_rate': 2.777777777777778e-06, 'epoch': 9.44}
{'loss': 0.6954, 'grad_norm': 3.3527462482452393, 'learning_rate': 1.388888888888889e-06, 'epoch': 9.72}
{'loss': 0.6939, 'grad_norm': 3.0110442638397217, 'learning_rate': 0.0, 'epoch': 10.0}


  0%|          | 0/450 [00:00<?, ?it/s]

{'eval_loss': 0.6933375597000122, 'eval_accuracy': 0.4915277777777778, 'eval_runtime': 224.1168, 'eval_samples_per_second': 32.126, 'eval_steps_per_second': 2.008, 'epoch': 10.0}
{'train_runtime': 32603.2927, 'train_samples_per_second': 8.833, 'train_steps_per_second': 0.552, 'train_loss': 0.6980445624457465, 'epoch': 10.0}


TrainOutput(global_step=18000, training_loss=0.6980445624457465, metrics={'train_runtime': 32603.2927, 'train_samples_per_second': 8.833, 'train_steps_per_second': 0.552, 'total_flos': 1.894399598592e+16, 'train_loss': 0.6980445624457465, 'epoch': 10.0})

In [9]:
results = trainer.evaluate()
print("Evaluation Results:", results)


sample_text = "I absolutely loved this product, it exceeded my expectations!"
encoded = baseline_tokenizer(sample_text, return_tensors="pt").to(device)
with torch.no_grad():
    output = model(**encoded)
    pred = output.logits.argmax(dim=-1).item()
    sentiment = "Positive" if pred == 1 else "Negative"
    print(f"Review: {sample_text}\nPredicted Sentiment: {sentiment}")

  0%|          | 0/450 [00:00<?, ?it/s]

Evaluation Results: {'eval_loss': 0.6930024027824402, 'eval_accuracy': 0.5084722222222222, 'eval_runtime': 180.8636, 'eval_samples_per_second': 39.809, 'eval_steps_per_second': 2.488, 'epoch': 10.0}
Review: I absolutely loved this product, it exceeded my expectations!
Predicted Sentiment: Positive
